In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from assistant.graph import graph

c:\Users\TK\Desktop\Projects\Deep_Researcher_App\api\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Collection 'in_memory_collection' created
 1. Characterization_of_forged_magnesium_alloys, Adding to vector store...
Time taken to add documents to vector store: 1.1 seconds


In [2]:
import uuid
thread_id = str(uuid.uuid4())

In [3]:
messages="How does texture affect the mechanical properties of magnesium alloys?"

In [5]:
import asyncio
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

asyncio.run(
    generate_stream()
    )

{'generate_query': {'search_query': 'effect of texture on mechanical properties of magnesium alloys'}}
{'document_search': {'sources_gathered': ['* Characterization of forged magnesium alloys : n/a\n* Characterization of forged magnesium alloys : n/a\n* Characterization of forged magnesium alloys : n/a'], 'research_loop_count': 1, 'document_search_results': ['Sources:\n\nSource Characterization of forged magnesium alloys:\n===\nURL: n/a\n===\nMost relevant content from source: # Characterization of forged magnesium alloys\n\n# 19\n\n|300 °C|20 μm|200 μm|\n|---|---|---|\n|350 °C|20 μm|200 μm|\n|400 °C|20 μm|200 μm|\n\nFig. 21 Micrographs of cast AZ80 alloy deformed at 300–400 °C at 0.001 s−1 to a strain of 1.0. Adapted from Prakash P, Toscano D, Shaha SK, Wells MA, Jahed H, Williams BW (2020) Effect of temperature on the hot deformation behavior of AZ80 magnesium alloy. Materials Science and Engineering A 794: 139923.\n\nThe axis represents a basal texture, while a more uniform distribu

# streaming

Streaming "updates"

In [6]:
from langchain_core.messages import AIMessageChunk


async def generate_stream_updates():
    async for token in graph.astream(
        {"research_topic": messages},  # Use the last message content as research topic
        {"configurable": {"thread_id": thread_id}},
        stream_mode="updates" 
    ): 
        
        if 'finalize_summary' in token.keys():
            running_summary = token["finalize_summary"]["running_summary"]        
            if 'generate_query' in token:
                yield f"data: {json.dumps({
                    'id': f'chat-{str(uuid.uuid4())}',
                    'object': 'chat.completion.chunk',
                    'created': int(time.time()),
                    'model': request.model,
                    'choices': [{
                        'index': 0,
                        'delta': {
                            'role': 'assistant',
                            'content': f"Question: {messages}\n\nAnswer: {running_summary}"
                        },
                        'finish_reason': None
                    }]
                })}\n\n" """
        

In [7]:
import asyncio
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

asyncio.run(
    generate_stream_updates()
    )

1
2
## Summary

Texture significantly influences the mechanical properties of magnesium alloys, particularly in their deformation behavior and strength. Research indicates that the texture of magnesium alloys, such as AZ31 and AZ80, evolves during processing, affecting their performance. In the AZ31 alloy, higher strains lead to a stronger texture; however, dynamically recrystallized (DRXed) grains exhibit lower texture intensity compared to un-recrystallized grains. Conversely, the AZ80 alloy shows that DRXed grains closely follow the texture of the deforming grains, indicating a more uniform texture development.

The strain rate also plays a crucial role, as increasing the strain rate enhances texture intensity, especially in the AZ80 alloy. The differences in aluminum content between alloys like AZ31 and AZ80 contribute to variations in texture development during deformation. Notably, studies have shown that un-recrystallized regions of the AZ91 alloy display a more pronounced basal

Streaming "messages

In [92]:
from langchain_core.messages import AIMessageChunk


async def generate_stream(iter=0, out_str=""):
    async for token in graph.astream(
        {"research_topic": messages},  # Use the last message content as research topic
        {"configurable": {"thread_id": thread_id}},
        stream_mode="messages" 
    ): 
        #print(type(token)) # <class 'tuple'>  stream_mode="messages" --> does acutal streaming 
        
        # print(token[0]) #content='' additional_kwargs={} response_metadata={} id='run-d875712c-f6a8-4cb0-9ace-4b04c0a8eb06'
        
        #print(token[1]) # {'thread_id': 'bd94e363-2788-44cd-8ef2-74a54871c773', 'langgraph_step': 1, 'langgraph_node': 'generate_query', 
            # 'langgraph_triggers': ['start:generate_query'], 'langgraph_path': ('__pregel_pull', 'generate_query'), 'langgraph_checkpoint_ns': 'generate_query:9caf1c19-ca93-b9c7-bf32-be7006d013e5', 
            # 'checkpoint_ns': 'generate_query:9caf1c19-ca93-b9c7-bf32-be7006d013e5', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': 0.0}
        
        # returns the thread_id --> token[1]["thread_id"])
        
        #print(f"iter:{iter} \n {token[0].content}, token length: {len(token[0].content)}\n")
        
        message_chunk = token[0].content
        message_metadata = token[1]        
        
        """ if '"query"' in message_chunk:
            print(message_chunk.replace('\n', ''), end='')
        elif '"aspect"' in message_chunk:
            print('\n' + message_chunk.replace('\n', ''), end='')
        else:
            print(message_chunk.replace('\n', ''), end='') """
        
        
        
        if isinstance(message_chunk, AIMessageChunk):
            if 'generate_query' == message_metadata['langgraph_node']:
                print(f"iter:{iter} \n {message_chunk}, token length: {len(message_chunk)}\n")


        
        """ # Stream intermediate research updates
        if isinstance(token, dict):
            # Stream each type of update with appropriate content
            if 'generate_query' in token:
                yield f"data: {json.dumps({
                    'id': f'chat-{str(uuid.uuid4())}',
                    'object': 'chat.completion.chunk',
                    'created': int(time.time()),
                    'model': request.model,
                    'choices': [{
                        'index': 0,
                        'delta': {
                            'role': 'assistant',
                            'content': f"Searching for: {token['generate_query']['search_query']}\n"
                        },
                        'finish_reason': None
                    }]
                })}\n\n" """
        

In [ ]:
import asyncio
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


asyncio.run(
    generate_stream()
    )